In [ ]:
import open3d as o3d
import numpy as np
from matplotlib import pyplot as plt 
import scipy
import time
import scipy.stats as st

Zero_time = time.perf_counter() 

Loading of both pointclouds

In [ ]:
print("Load a pts point cloud")
reference = o3d.io.read_point_cloud(r"K:\Projects\2025-02 Project BAEKELAND MEETHET\7.Recordings\2020-11-MEET_HET_HQ\Pointclouds\Wall\P30.pts")
t1 = time.perf_counter()
dt1 = t1 - Zero_time
print("Reference cloud loaded in %s s" % dt1)
print(reference)
target = o3d.io.read_point_cloud(r"K:\Projects\2025-02 Project BAEKELAND MEETHET\7.Recordings\2020-11-MEET_HET_HQ\Pointclouds\Wall\VLX.pts")
t2 = time.perf_counter()
dt2 = t2 - t1
print("Target cloud loaded in %s s" % dt2)
print(target)


Downsampling of the pointclouds

In [ ]:
voxel_size = 0.005

print("Downsampling Pointclouds to excalerate computations Voxel size: %s m" % voxel_size)

ref_voxel = reference.voxel_down_sample(voxel_size)
t3 = time.perf_counter()
dt3 = t3 - t2
print("Downsampled the pointclouds in %s s" % dt3)
print(ref_voxel)

tar_voxel = target.voxel_down_sample(voxel_size)

t4 = time.perf_counter()
dt4 = t4 - t3
print("Downsampled the pointclouds in %s s" % dt4)
print(tar_voxel)

In [ ]:
c2c = tar_voxel.compute_point_cloud_distance(ref_voxel)
t5 = time.perf_counter()
dt5 = t5 - t4
c2cArray = np.asarray(c2c)
print("Cloud 2 cloud calculated in %s s" % dt5)
print(c2cArray)

In [ ]:
tar_voxel.paint_uniform_color([0.5,0.5,0.5]) #Grey

In [ ]:
i =0
LOA30 = []
LOA20 = []
LOA10 = []
LOA00 = []
Outlier = []
Outlier_points = []
LOA30_points = []
LOA20_points = []
LOA10_points = []
while i < len(c2c):
    if c2c[i] <= 0.015:
        np.asarray(tar_voxel.colors)[i] = [0,1,0] #Green
        LOA30.append(np.append(tar_voxel.points[i],c2c[i]))
        LOA00.append(c2c[i])
        LOA30_points.append(tar_voxel.points[i])
    elif c2c[i] <= 0.05:
        np.asarray(tar_voxel.colors)[i] = [1,0.5,0] #Orange
        LOA20.append(np.append(tar_voxel.points[i],c2c[i]))
        LOA00.append(c2c[i])
        LOA20_points.append(tar_voxel.points[i])
    elif c2c[i] <= 0.1:
        np.asarray(tar_voxel.colors)[i] = [1,0,0] #Red
        LOA10.append(np.append(tar_voxel.points[i],c2c[i]))
        LOA00.append(c2c[i])
        LOA10_points.append(tar_voxel.points[i])
    elif c2c[i] > 0.1:
        Outlier.append(np.append(tar_voxel.points[i],c2c[i]))
        Outlier_points.append(tar_voxel.points[i])
    i = i+1

# print(np.asarray(LOA30))
# print(np.asarray(LOA20))
# print(np.asarray(LOA10))
# print(np.asarray(Outlier))
# print(np.asarray(Outlier_points))
t6 = time.perf_counter()
dt6 = t6 - t5
c2cArray = np.asarray(c2c)
print("LOAs seperated calculated in %s s" % dt6)

In [ ]:
Outlier_pcd = o3d.geometry.PointCloud()
Outlier_pcd.points = o3d.utility.Vector3dVector(Outlier_points)
print(np.asarray(Outlier_pcd))

In [ ]:
Outlier_pcd.paint_uniform_color([1,0,0]) #Red
o3d.visualization.draw_geometries([Outlier_pcd])


In [ ]:
LOA30_pcd = o3d.geometry.PointCloud()
LOA30_pcd.points = o3d.utility.Vector3dVector(LOA30_points)
print(np.asarray(LOA30_pcd))

In [ ]:
LOA30_pcd.paint_uniform_color([0,0,1]) #Red
o3d.visualization.draw_geometries([LOA30_pcd])

In [ ]:
y, x = np.histogram(c2cArray,bins= np.histogram_bin_edges(c2cArray,bins = 'auto'))
y = np.cumsum(y) / len(c2cArray)
fig, ax = plt.subplots()
ax.plot(x[:-1], y)
fig.show()

In [ ]:
y, x = np.histogram(c2cArray,bins= np.histogram_bin_edges(c2cArray,bins = 'auto', range = (0,0.1)))
y = np.cumsum(y) / len(c2cArray)
fig, ax = plt.subplots()
ax.plot(x[:-1], y)
fig.show()

In [ ]:
  nr_points_total = len(c2c)
  nr_points_LOA30 = len(LOA30)
  nr_points_LOA20 = len(LOA20)
  nr_points_LOA10 = len(LOA10)
  nr_points_LOA00 = len(LOA00)
  nr_points_Outlier = len(Outlier)

  prct_LOA30 = nr_points_LOA30/nr_points_total *100
  prct_LOA20 = nr_points_LOA20/nr_points_total *100
  prct_LOA10 = nr_points_LOA10/nr_points_total *100
  prct_Outlier = nr_points_Outlier/nr_points_total *100

  print("Percentages:")
  print("LOA30: %s %%" % prct_LOA30)
  print("LOA20: %s %%" % prct_LOA20)
  print("LOA10: %s %%" % prct_LOA10)
  print("Outliers: %s %%" % prct_Outlier)


In [ ]:
  prct_LOA30 = nr_points_LOA30/(nr_points_LOA00) *100
  prct_LOA20 = nr_points_LOA20/(nr_points_LOA00) *100
  prct_LOA10 = nr_points_LOA10/(nr_points_LOA00) *100

  print("Percentages (no outliers):")
  print("LOA30: %s %%" % prct_LOA30)
  print("LOA20: %s %%" % prct_LOA20)
  print("LOA10: %s %%" % prct_LOA10)

In [ ]:
percentile95 = np.percentile(c2cArray, 95)
print(percentile95)

In [ ]:
percentile95 = np.percentile(LOA00, 95)
print(percentile95)

In [ ]:
percentile68 = np.percentile(c2cArray,68)
print(percentile68)

In [ ]:
percentile68 = np.percentile(LOA00, 68)
print(percentile68)